In [ ]:
# !pip install clarifai
# !pip install openai
from clarifai_grpc.channel.clarifai_channel import ClarifaiChannel
from clarifai_grpc.grpc.api import resources_pb2, service_pb2, service_pb2_grpc
from clarifai_grpc.grpc.api.status import status_code_pb2

import openai

import dotenv
import os

dotenv.load_dotenv()

OPENAI_YOUR_KEY = os.getenv("CHATGPT_API_KEY")
openai.api_key = OPENAI_YOUR_KEY

PAT = os.getenv("CLARIFAI_API_KEY")
USER_ID = 'clarifai'
APP_ID = 'main'
MODEL_ID = 'general-image-recognition'
MODEL_VERSION_ID = 'aa7f35c01e0642fda5cf400f543e7c40'
IMAGE_URL = "{{ imageUrl }}"

In [ ]:
message = "해당 단어를 결합해 감성적인 문장을 만들어줘. 명상할 때 읽어줄 문장이고 해당 단어는 그날 명상하는 사람이 찍은 사진에 나온 단어야. 그 사람의 입장을 생각해서 만들어줘. 모든 단어를 활용할 필요는 없어. 이런 느낌을 느낄 수 있지 않았을까 하는 공감대를 형성하는 방향으로 문장을 작성해줘. 단어 뒤에 나온 숫자는 사진에서 자동으로 인식한 단어의 인식률이야. 그리고 영어로 답변해줘"

channel = ClarifaiChannel.get_grpc_channel()
stub = service_pb2_grpc.V2Stub(channel)

metadata = (('authorization', 'Key ' + PAT),)

userDataObject = resources_pb2.UserAppIDSet(user_id=USER_ID, app_id=APP_ID)

post_model_outputs_response = stub.PostModelOutputs(
    service_pb2.PostModelOutputsRequest(
        user_app_id=userDataObject,
        model_id=MODEL_ID,
        version_id=MODEL_VERSION_ID, 
        inputs=[
            resources_pb2.Input(
                data=resources_pb2.Data(
                    image=resources_pb2.Image(
                        url=IMAGE_URL
                    )
                )
            )
        ]
    ),
    metadata=metadata
)

if post_model_outputs_response.status.code != status_code_pb2.SUCCESS:
    print(post_model_outputs_response.status)
    raise Exception("Post model outputs failed, status: " + post_model_outputs_response.status.description)

output = post_model_outputs_response.outputs[0]

print("Predicted concepts:")
for concept in output.data.concepts:
  print("%s %.2f" % (concept.name, concept.value))
  message += concept.name + " " + str(concept.value) + "\n"

In [ ]:
# # 사용 모델을 설정합니다. chat GPT는 gpt-3.5-turbo를 사용합니다.
MODEL = "gpt-3.5-turbo"
USER_INPUT_MSG = message
message2 = ""

response = openai.ChatCompletion.create(
    model=MODEL,
    messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": USER_INPUT_MSG},
        # {"role": "assistant", "content": "Who's there?"},
    ],
    temperature=0.5,
    max_tokens=600  # 최대 토큰 수를 설정하여 답변의 길이를 제어합니다.
)

message2 += response['choices'][0]['message']['content']

# response['choices'][0]['message']['content']

In [ ]:
message2 += " 이걸 한국말로 번역해줘."

In [ ]:
USER_INPUT_MSG2 = message2
MODEL2 = "gpt-3.5-turbo-0613"

response2 = openai.ChatCompletion.create(
    model=MODEL2,
    messages=[
        {"role": "system", "content": "You are a kind assistant."},
        {"role": "user", "content": USER_INPUT_MSG2},
        {"role": "assistant", "content": "You must answer continuously."},
    ],
    temperature=0.6,
    max_tokens=600  # 최대 토큰 수를 설정하여 답변의 길이를 제어합니다.
)

response2['choices'][0]['message']['content']